In [1]:
import os

In [2]:
%pwd

'd:\\Opencv_projects\\chest_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Opencv_projects\\chest_classification'

In [ ]:
setx MLFLOW_TRACKING_URI "https://dagshub.com/aryan32134hello/chest_classification_mlflow_dvc.mlflow"

setx MLFLOW_TRACKING_USERNAME "aryan32134hello"

setx MLFLOW_TRACKING_PASSWORD "9d5f27a9d74d96bcd454246a1826fd97ee5ce6a6"

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/aryan32134hello/chest_classification_mlflow_dvc.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "aryan32134hello"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "9d5f27a9d74d96bcd454246a1826fd97ee5ce6a6"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int

In [9]:
from src.CNNChestClassifier.constants import *
from src.CNNChestClassifier.utils.common import create_directories,read_yaml,save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) ->EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/Chest-CT-Scan-data"),
            mlflow_uri="https://dagshub.com/aryan32134hello/chest_classification_mlflow_dvc.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

In [11]:
import os
import urllib.request as requests
from zipfile import ZipFile
import tensorflow as tf
import keras
import time
import mlflow
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config = config
    
    def train_valid_split(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.train_valid_split()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("scores.json"),data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss":self.score[0],"accuracy":self.score[1]}
            )

            if tracking_url_type_store!="file":
                mlflow.sklearn.log_model(self.model, "model", registered_model_name="ChestClassificationModel")
            else:
                mlflow.sklearn.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-07-16 01:08:48,213: INFO: common: yaml file: config\config.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1ooEK-e37Ig48zPBvi27d2AY9lhN34EPW/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5'}}
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1ooEK-e37Ig48zPBvi27d2AY9lhN34EPW/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_mo

C:\Users\Aryan Kumar\AppData\Roaming\Python\Python38\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ChestClassificationModel' already exists. Creating a new version of this model...
2024/07/16 01:09:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ChestClassificationModel, version 2
Created version '2' of model 'ChestClassificationModel'.
